ExE Interaction Analysis with NHAMES Data
-----------------------------------------


<h4>Analysis 1: Main Effect Filter</h4>
<ol>
<li>Create the Discovery and Replicate Datasets</li>
<li>Define Covariantes and Outcome</li>
<li>Run ExE Interaction to Discovery Dataset</li>
<li>Run Correct_PValue Function</li>
<li>Filter LRT_PVALUE_FDR < 0.1</li>
<li>Get Terms Pairs result after Filter</li>
-------------------------------------------
<li>Loop each Term Pair filted</li>
<li>Run ExE Interaction to Replicate Datase with each pair</li>
.....
</ol>

<h4>Analysis 2: Knowlegde Drive Filter</h4>
<ol>
<li>Create the Discovery and Replicate Datasets</li>
</ol>


<h3>Analysis 1: Main Effect Filter</h3>

In [2]:
# Packages
import os
import django
import pandas as pd
import igem
from pathlib import Path

rpy2 ModuleSpec(name='rpy2', loader=<_frozen_importlib_external.SourceFileLoader object at 0x137020520>, origin='/Users/andrerico/DEV/tests/chk_igem/venv/lib/python3.10/site-packages/rpy2/__init__.py', submodule_search_locations=['/Users/andrerico/DEV/tests/chk_igem/venv/lib/python3.10/site-packages/rpy2'])


In [3]:
# Set up Django environment
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "igem.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

# Variables
v_path_current = Path.cwd()

In [4]:
# Load NHAMES File with exposomes
# Here we are load Discovery and Replicate dataset filted by year
path_nhames_disc = str(v_path_current) + "/Analysis_1_discovery_FDR.txt"
path_nhames_repl = str(v_path_current) + "/Analysis_1_replication_FDR.txt"
df_nhames_disc = igem.epc.load.from_tsv(path_nhames_disc)
df_nhames_repl = igem.epc.load.from_tsv(path_nhames_repl)

Loaded 9,063 observations of 63 variables
Loaded 9,874 observations of 63 variables


In [13]:
# Setup columns rules
# list of Outcomes
list_outcome = ["logBMI"]

# list of Covariants
list_covariant = [
    "female",
    "black",
    "mexican",
    "other_hispanic",
    "other_eth",
    "SDDSRVYR",
    "SES_LEVEL",
    "RIDAGEYR",
]

In [57]:
# Run Interation Study to Discovery Data
df_interactions_disc = igem.epc.analyze.interaction_study(
    data=df_nhames_disc,
    outcomes=list_outcome,
    # covariates=list_covariant,
)

InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,063 of 9,063 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1,891 interactions
-------------------------
Running 1,891 interactions using 12 processes...


	Finished Running 1,891 interactions
42 tests had an error
	LBXVCF:URXUTL = NULL due to: too few complete observations (min_n filter: 102 < 200)
	LBXVCF:SMD057 = NULL due to: too few complete observations (min_n filter: 155 < 200)
	LBXVCF:URXP24 = NULL due to: too few complete observations (min_n filter: 139 < 200)
	LBXVCF:URXUPT = NULL due to: too few complete observations (min_n filter: 102 < 200)
	LBXVTO:URXUTL = NULL due to: too few complete observations (min_n filter: 102 < 200)
	LBXVTO:URXP24 = NULL due to: too few complete observations (min_n filter: 134 < 200)
	LBXVTO:URXUPT = NULL due to: too few complete observations (min_n filter: 102 < 200)
	URXUTL:LBXD05 = NULL due to: too few complete observations (min_n filter: 0 < 200)
	URXUTL:LBXF04 = NULL due to: too few complete observations (min_n filter: 0 < 200)
	URXUTL:LBXHXC = NULL due to: too few complete observations (min_n filter: 0 < 200)
	URXUTL:LBX170 = NULL due to: too few complete observations (min_n filter: 0 < 200)
	UR

In [58]:
df_interactions_disc = df_interactions_disc[df_interactions_disc['LRT_pvalue'] < 0.1]
df_corrested_pvalues_disc = igem.epc.analyze.add_corrected_pvalues(
    data=df_interactions_disc,
    pvalue="LRT_pvalue",
)
# TODO: This return None!

In [19]:
# Filter to keep only LRT_PVALUE_FDR < 0.1
df_fdr_disc = df_interactions_disc[df_interactions_disc['LRT_pvalue'] < 0.1]

In [22]:
# Get only Term1 and Term2 to run Interaction in Replicate Dataset
df_fdr_disc.reset_index(inplace=True)
df_exposome = igem.epc.modify.colfilter(df_fdr_disc, only=('Term1', 'Term2'))

Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 10 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 7 continuous variables
	2 of 3 unknown variables


Start ExE Interactions on Replicate Dataset

==> Use ExE Pairwise from Discovery Interactios saved in ls_exposomes

In [49]:
ls_inter = []
for e in df_exposome.itertuples():
    e1 = e[1]
    e2 = e[2]
    # Run Interation Study to Replicate Data
    df_inter = igem.epc.analyze.interaction_study(
        data=df_nhames_repl,
        outcomes=list_outcome,
        interactions=[(e1, e2)],
        # covariates=list_covariant,
    )
    df_inter.reset_index(inplace=True)
    # Save as List to High Performance
    ls_inter.append(
        [
            df_inter.iloc[0][0],
            df_inter.iloc[0][1],
            df_inter.iloc[0][2],
            df_inter.iloc[0][3],
            df_inter.iloc[0][4],
            df_inter.iloc[0][5],
            df_inter.iloc[0][6],
            df_inter.iloc[0][7],
            df_inter.iloc[0][8],
        ]
    )


InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,874 of 9,874 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...


	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for logBMI

Completed association study
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,874 of 9,874 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for logBMI

Completed association study
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,874 of 9,874 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed In

In [51]:
# Return to DataFrame
df = pd.DataFrame(ls_inter, columns=['Term1', 'Term2', 'Outcome', 'Conv', 'N', 'Beta', 'SE', 'Beta_pvalue', 'LRT_pvalue'])
print(df)

                 Term1     Term2 Outcome  Conv     N  Beta  SE  Beta_pvalue  \
0                LBXHA    LBXHPE  logBMI  True  1405   NaN NaN          NaN   
1                LBXHA    LBXCRY  logBMI  True  9205   NaN NaN          NaN   
2               LBXRBF  DR1TMFAT  logBMI  True  8970   NaN NaN          NaN   
3    cigarette_smoking    LBXHPE  logBMI  True   866   NaN NaN          NaN   
4               LBXRBF  DR1TM181  logBMI  True  8970   NaN NaN          NaN   
..                 ...       ...     ...   ...   ...   ...  ..          ...   
452             LBXRPL  DR1TTFAT  logBMI  True  8689   NaN NaN          NaN   
453             LBXCOT  DR1TM181  logBMI  True  8945   NaN NaN          NaN   
454             LBXVTO    LBXHXC  logBMI  True   760   NaN NaN          NaN   
455             LBXD05    LBXALC  logBMI  True  1329   NaN NaN          NaN   
456           DR1TMFAT    LBX194  logBMI  True  1249   NaN NaN          NaN   

       LRT_pvalue  
0    3.219465e-03  
1    4.7073

<h3>Analysis 2: Knowledge Filter</h3>

<p>If start in Analysis 2, run the two first blocks.</p>

In [1]:
# Packages
import os
import django
import pandas as pd
import igem
from pathlib import Path

rpy2 ModuleSpec(name='rpy2', loader=<_frozen_importlib_external.SourceFileLoader object at 0x13cbde470>, origin='/Users/andrerico/DEV/tests/chk_igem/venv/lib/python3.10/site-packages/rpy2/__init__.py', submodule_search_locations=['/Users/andrerico/DEV/tests/chk_igem/venv/lib/python3.10/site-packages/rpy2'])


In [2]:
# Set up Django environment
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "igem.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

# Variables
v_path_current = Path.cwd()

In [55]:
# Load NHAMES File with exposomes
# Here we are load Discovery and Replicate dataset filted by year
path_nhames_disc = str(v_path_current) + "/Analysis_2_discovery_all_bmi.txt"
path_nhames_repl = str(v_path_current) + "/Analysis_2_replication_all_bmi.txt"
path_nhames_description = str(v_path_current) + "/nhanes_description.csv"

df_nhames_disc_2 = igem.epc.load.from_tsv(path_nhames_disc)
df_nhames_repl_2 = igem.epc.load.from_tsv(path_nhames_repl)
df_nhames_description = igem.epc.load.from_csv(path_nhames_description)

Loaded 9,063 observations of 362 variables
Loaded 9,874 observations of 362 variables
Loaded 3,174 observations of 36 variables


In [57]:
# Get NHAMES Columns ID and Description
df_nhames_description.reset_index(inplace=True)
# Convert description values to lowercase
# This is important to IGEM Search Enginer
df_nhames_description['var_desc_low'] = df_nhames_description['var_desc'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_nhames_description['var_low'] = df_nhames_description['var'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df_nhames_description.drop_duplicates(inplace=True, subset=['var'])
df_nhames_words = igem.epc.modify.colfilter(df_nhames_description, only=('var', 'var_low', 'var_desc', 'var_desc_low'))

# backup (transf all in lower case)
# df_nhames_description_lower = df_nhames_description.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Running colfilter
--------------------------------------------------------------------------------
Keeping 4 of 40 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 27 continuous variables
	4 of 13 unknown variables


In [59]:
# Keep in NHAMES Description only NHAMES ID that we have in the "NHAMES File with exposomes"

# 1. get columns from "NHAMES File with exposomes"
ls_expos_columns = df_nhames_disc_2.columns
# 2. Convert all items to lowercase using list comprehension
ls_expos_columns = [item.lower() for item in ls_expos_columns]
print(f"We have {len(ls_expos_columns)} columns in NHAMES File Data")

# 3. Find in the NHAMES Descrition all columns we have in NHAMES Files with Exposomes
#    Here we will use ID (var) as key and is import normaliz all to lower case to match
df_nhames_words_filted = df_nhames_words[df_nhames_words['var_low'].isin(ls_expos_columns)]
print(f"We have {len(df_nhames_words)} NHAMES IDs on Description Files")
print(f"  and we found {len(df_nhames_words_filted)} matches with NHAMES Files Data")

# 4. Find the elements not present in the DataFrame
elements_not_in_df = [element for element in ls_expos_columns if element not in df_nhames_words_filted['var_low'].values]
print(f"  the {len(elements_not_in_df)} not matches are:")
print(elements_not_in_df)


We have 362 columns in NHAMES File Data
We have 1151 NHAMES IDs on Description Files
  and we found 353 matches with NHAMES Files Data
  the 9 not matches are:
['ses_level', 'sddsrvyr', 'female', 'black', 'mexican', 'other_hispanic', 'other_eth', 'ridageyr', 'logbmi']


In [60]:
# Use the var_desc field in NHAMES Description to Search Terms on GE.db
# TODO: Improve the word_to_term() to receive a DF and witch column used to search
# 1. save only var_desc in csv file
df_nhames_words_filted[['var_desc_low']].to_csv(str(str(v_path_current) + "/nhames_words.csv"), index=False)

In [10]:
# Run IGEM Search Enginer
v_check = igem.ge.filter.word_to_term(str(str(v_path_current) + "/nhames_words.csv"))
if v_check:
    print("Search ran with success")

Start: Search for WORDS to term
    Rows to process: 353 rows
    Rows on KeyWord: 2188567 rows
File with the results sucessfully created in /users/andrerico/dev/tests/chk_igem/script_nhames/output_word_to_term.csv
Search ran with success


In [11]:
# Run IGEM Term Map Relationship
df_terms = pd.read_csv(str(str(str(v_path_current) + "/output_word_to_term.csv")))
ls_terms = df_terms['term'].tolist()
df_terms_pair = igem.ge.filter.term_map(term=ls_terms)
print(f"Found {len(df_terms_pair)} terms relationships")

GE.filter run successfully and returned 169571 relationships
Your GE.db Version Tag is: GE.db-TAG:37-38-39-44-41-31-30-29-19-20-18-45


In [61]:
# Keep only Pairs where both in NHAMES List
# 1. Keep only Terms and aggregate it
df_terms_pair_clean = igem.epc.modify.colfilter(df_terms_pair, only=["term_1", "term_2"])
df_terms_pair_clean = df_terms_pair_clean.drop_duplicates(keep="last")

# 2. Merge IGEM Term with the source NHAMES Strings ()
# TODO: we can use IGEM to Merge as well.
df_terms_pair_clean = df_terms_pair_clean.merge(
    df_terms, left_on="term_1", right_on="term", how="left"
)
df_terms_pair_clean = df_terms_pair_clean.merge(
    df_terms, left_on="term_2", right_on="term", how="left"
)

# 3. Clean the Dataframe
df_terms_pair_clean = igem.epc.modify.colfilter(df_terms_pair_clean, only=["term_1", "term_2", 'string_x', 'string_y'])
df_terms_pair_clean = df_terms_pair_clean.dropna()
df_terms_pair_clean = df_terms_pair_clean.reindex(columns=['term_1', 'string_x', 'term_2', 'string_y'])
df_terms_pair_clean = df_terms_pair_clean.reset_index() 
df_terms_pair_clean = df_terms_pair_clean.rename(
    columns={"string_x": "Exposome_1", "string_y": "Exposome_2"}
)
print(len(df_terms_pair_clean))


Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 13 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 3 continuous variables
	2 of 10 unknown variables
Running colfilter
--------------------------------------------------------------------------------
Keeping 4 of 20 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 2 continuous variables
	4 of 18 unknown variables
4168


In [63]:
# 4. Merge NHAMES Scrings with NHAMES ID and Description (var and var_desc)

df_nhames_map = df_terms_pair_clean.merge(
    df_nhames_words_filted, left_on="Exposome_1", right_on="var_desc_low", how="left"
)

df_nhames_map = df_nhames_map.merge(
    df_nhames_words_filted, left_on="Exposome_2", right_on="var_desc_low", how="left"
)
print()
# 5. Drops columns unnecessary and normalize columns
df_nhames_pairs = igem.epc.modify.colfilter(
    df_nhames_map,
    only=["term_1", "term_2", "Exposome_1", "Exposome_2", "var_x", "var_y"]
    )
df_nhames_map = df_nhames_map.rename(
    columns={"Exposome_1": "nhames_desc_1", "Exposome_2": "nhames_desc_2", "var_x": "nhames_id_1", "var_y": "nhames_id_2"}
)
df_nhames_map = df_nhames_map.reindex(columns=['term_1', 'nhames_id_1', "nhames_desc_1", 'term_2', 'nhames_id_2', "nhames_desc_2"])


Running colfilter
--------------------------------------------------------------------------------
Keeping 6 of 13 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 1 continuous variables
	6 of 12 unknown variables


<p>Run the ExE Interation on the list</p>

In [ ]:
# Setup columns rules
# list of Outcomes
list_outcome = ["logBMI"]

# list of Covariants
list_covariant = [
    "female",
    "black",
    "mexican",
    "other_hispanic",
    "other_eth",
    "SDDSRVYR",
    "SES_LEVEL",
    "RIDAGEYR",
]

In [66]:
# Loop on nhames map 
ls_inter = []
for e in df_nhames_map.itertuples():
    e1 = e[2]
    e2 = e[5]
    print(e1, ' ', e2)
    if e1 == e2:
        continue
    # Keeo only columns to run regression
    df_interactions = df_nhames_disc_2.loc[
            :, list_covariant + list_outcome + list([e1, e2])
        ]
    # TODO: we need treat NaN?
    df_interactions = df_interactions.fillna(0)
    df_inter = igem.epc.analyze.interaction_study(
            data=df_interactions,
            outcomes=list_outcome,
            interactions=[(e1, e2)],
            covariates=list_covariant,
        )
    df_inter.reset_index(inplace=True)
    # Save as List to High Performance
    ls_inter.append(
        [
            df_inter.iloc[0][0],
            df_inter.iloc[0][1],
            df_inter.iloc[0][2],
            df_inter.iloc[0][3],
            df_inter.iloc[0][4],
            df_inter.iloc[0][5],
            df_inter.iloc[0][6],
            df_inter.iloc[0][7],
            df_inter.iloc[0][8],
        ]
    )

# Return to DataFrame
df_result = pd.DataFrame(ls_inter, columns=['Term1', 'Term2', 'Outcome', 'Conv', 'N', 'Beta', 'SE', 'Beta_pvalue', 'LRT_pvalue'])
print(df_result)

LBXPFOA   LBXHBC
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,063 of 9,063 observations
Regressing 2 variables
	0 binary variables
	0 categorical variables
	2 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...


	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for logBMI

Completed association study
LBXPFOA   LBXHBS
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,063 of 9,063 observations
Regressing 2 variables
	0 binary variables
	0 categorical variables
	2 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for logBMI

Completed association study
LBXPFDE   LBXHBC
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,063 of 9,063 observations
Regressing 2 variables
	0 binary variables
	0 categorical variables
	2 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 t

<h4> Task to Do </h4>
<ol>
<li>Now we need filter the results</li>
<li>select pairs</li>
<li>Run regression on Replicate Dataset</li>
</ol>